### Statistics Cunhaobot

First, import the tools we need.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import json

Then clean a bit the json file containing all the information.

In [ ]:
results = []
filename = 'bq-results-20190624-222826-z9jyo1950ix9.json'
with open(filename, encoding='latin-1') as f:
    for line in f.read().strip().split("\n"):
        results.append(json.loads(line)['textPayload'])


clean_results = []
for result in results:
    if not result.endswith("}"):
        continue
    if not result.startswith("{"):
        result = result[result.index("{"):]
    clean_results.append(result)


#slow and dangerous
results_dicts = [eval(result) for result in clean_results]

And trim a little more the file to save the things we want to get the statistics.

First, we get the 'query_inline_results', those results contain all the information we want: Username (or first name), query type (if the query is long, short, audio...), user_id (Telegram user id) and the query data the user used or submited; if something is not found is replaced by "EMPTY" (or "Maybe" (2) in the case if the user is a bot or not... You can't be 100% sure).

Lastly, create a dictionary with all this information to get Pandas DataFrame working.

In [ ]:
inline_results = [msg for msg in results_dicts if 'chosen_inline_result' in msg.keys()]
usernames = []
list_queries = []

for idx in range(len(inline_results)):
    try:
        user = inline_results[idx]['chosen_inline_result']['from']['username']
    except KeyError:
        user = inline_results[idx]['chosen_inline_result']['from']['first_name']
    user_id = inline_results[idx]['chosen_inline_result']['from']['id']
    try:
        is_bot = inline_results[idx]['chosen_inline_result']['from']['is_bot']
    except KeyError:
        is_bot = "Maybe"
    try:
        query_type = inline_results[idx]['chosen_inline_result']['query']
    except KeyError:
        query_type = "EMPTY"
    try: 
        query_data = inline_results[idx]['chosen_inline_result']['result_id']
    except KeyError:
        query_data = "EMPTY"
    list_queries.append((user_id, user, is_bot, query_type, query_data))
    if (user not in usernames):
        usernames.append(user)
        
query_dict = {}
queries = []
lines = []
user_checked = []
uses = [user[1] for user in list_queries]
id_user = 0
for tuple_query in list_queries:
    current_user_id = tuple_query[0]
    current_user = tuple_query[1]
    current_user_is_bot = tuple_query[2]
    if current_user not in user_checked:
        n_uses = len([idx for idx in range(len(uses)) if uses[idx] == current_user])
        query_types = [query[3] for query in list_queries if current_user == query[1]]
        queries_data = [line[4] for line in list_queries if current_user == line[1]]
        user_checked.append(current_user)
        query_dict[id_user] = {'user_id': current_user_id, 'user': current_user, 
                              'is_bot': current_user_is_bot, 'query_type': query_types, 
                              'query_data': queries_data, 'num_uses': n_uses}
        id_user += 1

Now the F-U-N begins, let's get the DataFrame and study the logs!

In [ ]:
dataframe_logs = pd.DataFrame(query_dict).transpose()
num_observations = len(dataframe_logs.index)
num_features = len(dataframe_logs.columns)

print("There are: {} observations and {} features right now".format(num_observations, num_features))

fig, ax = plt.subplots()
N, bins, patches = ax.hist(x=dataframe_logs["is_bot"], edgecolor='black', linewidth=2)
plt.ylabel("Frecuency")
plt.xlabel("Is bot")
plt.grid()
for idx in range(0,3): patches[idx].set_facecolor('crimson')
for idx in range(3,7): patches[idx].set_facecolor('aqua')
for idx in range(7,len(patches)): patches[idx].set_facecolor('chartreuse')
true_data = mpatches.Patch(color='aqua', label='True')
false_data = mpatches.Patch(color='crimson', label='False')
maybe_data = mpatches.Patch(color='chartreuse', label='Maybe')
plt.legend(handles=[false_data, true_data, maybe_data], loc='upper left')
plt.title("Count of bots using this bot")
plt.show()